In [3]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
import fiftyone.utils.data as foud
import fiftyone.utils.random as four
from fiftyone import ViewField as F
import csv
import pandas as pd
import torch.cuda
from os.path import join
import json
import copy
from tqdm import tqdm

In [4]:
torch.cuda.is_available()

True

In [5]:
!pwd

/home/person/Laletin_Diplom/ptz-camera/notebooks/data_manipulation


In [6]:
with open("../../data/raw/MIAP_FULL/metadata/open_images_extended_miap_boxes_train.csv") as f:
    reader = csv.reader(f, delimiter=',')
    det_data = [row for row in reader]

In [7]:
with open("../../data/raw/MIAP/metadata/classes.csv") as f:
    reader = csv.reader(f, delimiter=",")
    classes = {row[0] : row[1] for row in reader}

In [8]:
def to_detections(sample: pd.DataFrame):
    detections = []
    
    label = sample["ImageID"].iloc[0]
    
    for _, row in sample.iterrows():
        bbox = [row["XMin"], row["YMin"], row["XMax"] - row["XMin"], row["YMax"] - row["YMin"]]
        detection = fo.Detection(bounding_box=bbox, label=row["LabelName"])
        
        detection["IsOccluded"] = bool(row["IsOccluded"])
        detection["IsTruncated"] = bool(row["IsTruncated"])
        detection["IsGroupOf"] = bool(row["IsGroupOf"])
        detection["IsDepictionOf"] = bool(row["IsDepictionOf"])
        detection["IsInsideOf"] = bool(row["IsInsideOf"])
        
        detections.append(detection)
    
    return label, fo.Detections(detections=detections)

In [9]:
df = pd.read_csv("../../data/raw/MIAP_FULL/metadata/open_images_extended_miap_boxes_train.csv")
df["LabelName"] = df["LabelName"].apply(lambda x: classes[x])
df

,ImageID,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepictionOf,IsInsideOf,GenderPresentation,AgePresentation
0,0d05f4fb7c2c80eb,Person,1,0.115625,0.964063,0.475000,0.993750,1,1,1,0,0,Unknown,Unknown
1,07961f9cae2d6646,Person,1,0.183976,0.286350,0.362637,0.578022,0,1,0,0,0,Predominantly Feminine,Middle
2,006a1bb8403536e4,Person,1,0.582812,0.957812,0.122917,0.970833,0,1,0,0,0,Predominantly Masculine,Middle
3,e937b900f636360b,Person,1,0.106038,0.910162,0.073009,1.000000,1,1,0,0,0,Unknown,Unknown
4,a6f42ca6f29954aa,Person,1,0.293944,0.558346,0.125828,0.997792,1,0,0,0,0,Predominantly Masculine,Middle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373381,7d7b302d11dc1474,Person,1,0.045313,0.592188,0.283333,0.972917,1,1,0,0,0,Unknown,Unknown
373382,b3d9da2c04e028e1,Person,1,0.773529,0.908824,0.636364,1.000000,1,0,0,0,0,Predominantly Masculine,Middle
373383,ec8b0bd4fa2075ae,Person,1,0.360206,0.716981,0.076046,0.629278,0,1,0,0,0,Unknown,Unknown
373384,db354a954b34e79a,Person,1,0.656342,0.806785,0.356195,0.860619,0,1,0,0,0,Predominantly Feminine,Middle


In [15]:
df = df[ # (df["IsOccluded"] == 0) & 
   (df["IsTruncated"] == 0) & 
   (df["IsGroupOf"] == 0) & 
   (df["IsDepictionOf"] == 0) & 
   (df["XMin"] != df["XMax"]) & 
   (df["YMin"] != df["YMax"])]

In [16]:
detections = df.groupby("ImageID", group_keys=True).apply(lambda x: to_detections(x))


/tmp/ipykernel_253499/3934255324.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  detections = df.groupby("ImageID", group_keys=True).apply(lambda x: to_detections(x))


In [17]:
dataset = fo.Dataset(name="MIAP_RAW_FULL", persistent=True, overwrite=True)

In [18]:
for label, detection in detections:
    image_path = f"../../data/raw/MIAP_FULL/data/{label}.jpg"
    sample = fo.Sample(filepath=image_path)
    sample["detections"] = detection
    dataset.add_sample(sample)

In [19]:
dataset.compute_metadata()

Computing metadata...
 100% |█████████████| 41499/41499 [12.8s elapsed, 0s remaining, 3.5K samples/s]      


In [ ]:
model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")
embeddings = dataset.compute_embeddings(model)

In [ ]:
results = fob.compute_visualization(
    dataset, embeddings=embeddings, seed=51, brain_key="img_viz"
)

In [ ]:
fob.compute_similarity(dataset, brain_key="image_sim")

In [ ]:
fob.compute_uniqueness(dataset, embeddings=embeddings, model=model)

In [ ]:
session = fo.launch_app(dataset, auto=False)
session.url

In [20]:
three_channels_view = dataset.match(F("metadata.num_channels") == 3)
splits = ["train", "val", "test"]

In [21]:
four.random_split(three_channels_view, {"train": 0.8, "val": 0.1, "test": 0.1}, seed=0)
train_view = dataset.match_tags("train")
val_view = dataset.match_tags("val")
test_view = dataset.match_tags("test")
for split in splits:
    split_view = three_channels_view.match_tags(split)
    split_view.export(
       export_dir=f"../../data/processed/MIAP_FULL_MOBILE_NET/",
       dataset_type=fo.types.COCODetectionDataset,
       label_field="detections"
    )
train_view.export(
   export_dir="../../data/processed/MIAP_FULL_MOBILE_NET",
   dataset_type=fo.types.COCODetectionDataset,
   label_field="detections"
)
val_view.export(
   export_dir="../../data/processed/MIAP_FULL_COCO_VAL",
   dataset_type=fo.types.COCODetectionDataset,
   label_field="detections"
)
test_view.export(
   export_dir="../../data/processed/MIAP_FULL_COCO_TEST",
   dataset_type=fo.types.COCODetectionDataset,
   label_field="detections"
)

 100% |█████████████| 32062/32062 [1.1m elapsed, 0s remaining, 550.6 samples/s]      
 100% |███████████████| 4008/4008 [10.5s elapsed, 0s remaining, 349.6 samples/s]      
 100% |███████████████| 4008/4008 [9.8s elapsed, 0s remaining, 485.9 samples/s]       


In [ ]:
export_dir = "../../data/processed/MIAP_FULL_YOLO"
classes=["Background", "Person"]
splits = ["train", "val", "test"]

for split in splits:
    split_view = three_channels_view.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split=split,
        classes=classes,
    )

In [22]:
for path in ["../../data/processed/MIAP_FULL_COCO_TRAIN", 
             "../../data/processed/MIAP_FULL_COCO_VAL", 
             "../../data/processed/MIAP_FULL_COCO_TEST"]:
    with open(join(path, "labels.json")) as labels:
        data = json.load(labels)
        new_data = copy.deepcopy(data)
        new_data["categories"] = [{'id': 0, 'name': 'Background', 'supercategory': None},
                              {'id': 1, 'name': 'Person', 'supercategory': None}]
        for annotation in tqdm(new_data['annotations']):
            image_id = annotation['image_id']
            image = new_data['images'][image_id - 1]
            # annotation['bbox'] = [annotation['bbox'][0] / image['width'],
            #                       annotation['bbox'][1] / image['height'],
            #                       annotation['bbox'][2] / image['width'],
            #                       annotation['bbox'][3] / image['height']]
            annotation['category_id'] = 1
            
    with open(join(path, "new_labels.json"), 'w') as output_labels:   
        json.dump(new_data, output_labels)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12065/12065 [00:00<00:00, 1703560.94it/s]
